In [1]:
import pandas as pd
ri=pd.read_csv('ri_statewide_2019_12_17.csv')
ri.columns
print(ri.head())
print(ri.isnull().sum())
print(ri.shape)

print(ri.isnull().sum())
ri.dropna(subset=['driver_gender'],inplace=True)
ri.dtypes

combined=ri.date.str.cat(ri.time,sep=' ')  #combined two columns together
ri['stop_datetime']=pd.to_datetime(combined)   #convert to datetime
ri.set_index('stop_datetime',inplace=True)  #set this new column as the index


#count the unique values of violation
print(ri.violation.count_values())
print(ri.violation.count_values(normalize=True))

#create a dateframe of female
female=ri[ri.driver_gender=='F']
#male
male=ri[ri.driver_gender=='M']
#proportions of violation for two genders
print(female.violation.value_counts(normalize=True))
print(male.violation.value_counts(normalize=True))


#dataframe for female speeding
female_and_speed=ri[(ri.driver_gender=='F')&(ri.violation=='Speeding')]
print(female_and_speeding.stop_outcome.value_counts(normalize=True))

#search rate  by value_counts/mean since search_conducted is a boolean
print(ri.search_conducted.value_counts(normalize=Ture))
print(ri.search_conducted.mean())

#compare the two genders'search rate
print(ri.groupby(['driver_gender']).search_conducted.mean())

#search_type values
print(ri.search_type.value_counts())
#see if 'search_type' has a string 'Protective frisk'
ri['frisk']=ri.search_type.str.contains('Protective Frisk',
                                       na=False)  #na value will be assigned as False in this case

print(ri.frisk.dtype)
print(ri.frisk.sum())


searched=ri[ri.search_conducted==True]
print(searched.frisk.mean())  #percentage of frisk
print(searched.groupby(['driver_gender']).frisk.mean())  #see the percentage of frisk by gender



hourly_arrest_rate=ri.groupby(ri.index.hour).is_arrested.mean()

import matploylib.pyplot as plt
hourly_arrest_rate.plot()
plt.xlabel('Hour')
plt.ylabel('Arrest Rate')
plt.title('Arrest Rate by Time of Day')
plt.show

#change duration from time slot to a specific time
print(ri.stop_duration.unique())
mapping={'0-15 Min':8, '16-30 Min':23,'30+ Min':45}
ri['stop_minutes']=ri.stop_duration.map(mapping)
print(ri.stop_minutes.unique())

stop_length=ri.groupby('violation_raw').stop_minutes.mean()
stop_length.sort_valuues().plot(kind='barh')
plt.show()

#Does weather influence police's searching behavior?
weather=pd.read_csv('weather.csv')
weather[['TMIN','TAVG','TMAX']].plot(kind='box')
plt.show()

weather['TDIFF']=weather.TMAX-weather.TMIN
print(weather['TDIFF'].describe())
weather.TDIFF.plot(kind='hist',bins=20)
plt.show()

#bad weather distribution
WT=weather.loc[:,'WT01':'WT22']  #all kinds of bad weathers
#calculate the sum of each row in 'WT'
weather['bad_conditions']=WT.sum(axis='columns')
#replace missing values with 0
weather['bad_conditions'] = weather.bad_conditions.fillna(0).astype('int')
weather.bad_conditions.plot(kind='hist')
plt.show()

# Count the unique values in 'bad_conditions' and sort the index
print(weather.bad_conditions.value_counts().sort_index())
# Create a dictionary that maps integers to strings
mapping = {0:'good', 1:'bad', 2:'bad', 3:'bad',4:'bad',5:'worse',
    6:'worse',7:'worse',8:'worse',9:'worse'}
# Convert the 'bad_conditions' integers to strings using the 'mapping'
weather['rating'] = weather.bad_conditions.map(mapping)

# Count the unique values in 'rating'
print(weather.rating.value_counts())

# Create a list of weather ratings in logical order
cats=['good','bad','worse']

# Change the data type of 'rating' to category
weather['rating'] = weather.rating.astype('category', ordered=True, categories=cats)

# Reset the index of 'ri'
ri.reset_index(inplace=True)

# Examine the head of 'ri'
print(ri.head())

# Create a DataFrame from the 'DATE' and 'rating' columns
weather_rating= weather[['DATE','rating']]

# Merge 'ri' and 'weather_rating' using a left join
ri_weather = pd.merge(left=ri, right=weather_rating, left_on='stop_date', right_on='DATE', how='left')
# Set 'stop_datetime' as the index of 'ri_weather'
ri_weather.set_index('stop_datetime', inplace=True)


# the arrest rate for each 'violation' and 'rating'
arrest_rate = ri_weather.groupby(['violation', 'rating']).is_arrested.mean()

# Print the arrest rate for moving violations in bad weather
print(arrest_rate['Moving violation']['bad'])

# Print the arrest rates for speeding violations in all three weather conditions
print(arrest_rate['Speeding'])

# Unstack the 'arrest_rate' Series into a DataFrame
print(arrest_rate.unstack())


/Users/liyidan/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,17,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   raw_row_number        date      time zone subject_race subject_sex  \
0               1  2005-11-22  11:15:00   X3        white        male   
1               2  2005-10-01  12:20:00   X3        white        male   
2               3  2005-10-01  12:30:00   X3        white      female   
3               4  2005-10-01  12:50:00   X3        white        male   
4               5  2005-10-01  13:10:00   X3        white      female   

  department_id       type arrest_made citation_issued  ... reason_for_stop  \
0           200  vehicular       False            True  ...        Speeding   
1           200  vehicular       False            True  ...        Speeding   
2           200  vehicular       False            True  ...        Speeding   
3           200  vehicular       False            True  ...        Speeding   
4           200  vehicular       False            True  ...        Speeding   

  vehicle_make vehicle_model raw_BasisForStop raw_OperatorRace  \
0          NaN      

AttributeError: 'DataFrame' object has no attribute 'violation'